In [1]:
# Deep Convolutional GANs

# Importing the libraries
# from __future__ import print_function
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

import numpy as np
from mpi4py import MPI

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Creating the network to create the peer2peer connection for swaping of the Discriminator

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

In [4]:
# Setting some hyperparameters
batchSize = 640 # We set the size of the batch.
imageSize = 64 # We set the size of the generated images (64x64).

In [5]:
# Creating the transformations
transform = transforms.Compose([transforms.Resize(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) # We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.
#nc = 3

In [6]:
# Loading the dataset
dataset = dset.CIFAR10(root = './data', download = True, transform = transform)

Files already downloaded and verified


In [7]:
# We download the training set in the ./data folder and we apply the previous transformations on each image.
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2) 


In [8]:
""" 
We use dataLoader to get the images of the training set batch by batch.
We ust the shuffle = True because we want to get the dataset in random order so that we can train model more precisely.
We use num_worker = 2 which represent the number of thread and the worker servers to define the 
"""

# Defining the weights_init function that takes as input a neural network m and that will initialize all its weights.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


In [9]:
# Defining the copy of the generator to shuffle between diffrent severs
def copyGenerator():
    layer_num = 0
    for param in netG.parameters():
        #print(rank, "started")
        if (rank == 0):
            data = param.data.numpy().copy()
            #print(rank, data.shape)
        else:
            data = None
            #print(rank, data.shape)

        #print(rank, "before bcast")
        #comm.Barrier()
        data = comm.bcast(data, root = 0)
        #print(rank, "after bcast")
        if (rank != 0):
            param.data = torch.from_numpy(data)
            #print("Node rank " + str(rank) + " has synched generator layer " + str(layer_num))

        layer_num += 1
        #comm.Barrier()

In [10]:
#Peer2Peer shuffling of the Discriminator
def shuffleDiscriminators():
    if (rank != 0):
        layer_num = 0
        for param in netD.parameters():
            outdata = param.data.numpy().copy()
            indata = None

            if (rank != size - 1):
                comm.send(outdata, dest=rank + 1, tag=1)
            if (rank != 1):
                indata = comm.recv(source = rank-1, tag=1)

            if (rank == size - 1):
                comm.send(outdata, dest=1, tag=2)
            if (rank == 1):
                indata = comm.recv(source = size - 1, tag=2)
            # Shuffling the Discriminator
            param.data = torch.from_numpy(indata)
            layer_num += 1

In [11]:
# Defining the generator

class G(nn.Module):

    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
            nn.Tanh()
        )

    def forward(self, input):
        output = self.main(input)
        return output

In [12]:
# Creating the generator
netG = G()
netG.to(device)
netG.apply(weights_init)

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [13]:
# Defining the discriminator

class D(nn.Module):

    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 512, 4, 2, 1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(512, 1, 4, 1, 0, bias = False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1)


In [14]:
# Creating the discriminator
netD = D()
netD.to(device)
netD.apply(weights_init)

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [15]:
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))

for epoch in range(3):

    if (epoch % 2 == 0):
        shuffleDiscriminators()

    for i, data in enumerate(dataloader, 775):
        
        # 1st Step: Updating the weights of the neural network of the discriminator

        netD.zero_grad()
        
        # Training the discriminator with a real image of the dataset
        real, _ = data
        input = Variable(real).to(device)
        target = Variable(torch.ones(input.size()[0])).to(device)
        output = netD(input).to(device)
        errD_real = criterion(output, target)
        
        # Training the discriminator with a fake image generated by the generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1)).to(device)
        fake = netG(noise).to(device)
        target = Variable(torch.zeros(input.size()[0])).to(device)
        output = netD(fake.detach()).to(device)
        errD_fake = criterion(output, target)
        
        # Backpropagating the total error
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()

        # 2nd Step: Updating the weights of the neural network of the generator

        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0])).to(device)
        output = netD(fake).to(device)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()
        
        # 3rd Step: Printing the losses and saving the real images and the generated images of the minibatch every 100 steps

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), errD.item(), errG.item()))
        if i % 100 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True)



[0/25][775/79] Loss_D: 1.7166 Loss_G: 6.5996
[0/25][776/79] Loss_D: 1.1719 Loss_G: 5.8026
[0/25][777/79] Loss_D: 1.4136 Loss_G: 6.1073
[0/25][778/79] Loss_D: 1.3693 Loss_G: 6.3615
[0/25][779/79] Loss_D: 1.4312 Loss_G: 6.9311
[0/25][780/79] Loss_D: 1.2808 Loss_G: 8.0111
[0/25][781/79] Loss_D: 0.8122 Loss_G: 7.2813
[0/25][782/79] Loss_D: 0.9929 Loss_G: 9.9526
[0/25][783/79] Loss_D: 0.6393 Loss_G: 8.0719
[0/25][784/79] Loss_D: 1.3316 Loss_G: 11.4278
[0/25][785/79] Loss_D: 0.6379 Loss_G: 8.5148
[0/25][786/79] Loss_D: 1.3275 Loss_G: 12.2510
[0/25][787/79] Loss_D: 0.4435 Loss_G: 9.6855
[0/25][788/79] Loss_D: 0.8021 Loss_G: 11.1497
[0/25][789/79] Loss_D: 0.3948 Loss_G: 9.3519
[0/25][790/79] Loss_D: 1.0408 Loss_G: 13.7260
[0/25][791/79] Loss_D: 0.4272 Loss_G: 10.6118
[0/25][792/79] Loss_D: 0.6541 Loss_G: 10.8621
[0/25][793/79] Loss_D: 0.3518 Loss_G: 10.5200
[0/25][794/79] Loss_D: 0.4639 Loss_G: 11.9108
[0/25][795/79] Loss_D: 0.2848 Loss_G: 9.7066
[0/25][796/79] Loss_D: 0.9734 Loss_G: 17.2355
[

[2/25][799/79] Loss_D: 0.2087 Loss_G: 5.2519
[2/25][800/79] Loss_D: 0.2427 Loss_G: 5.4700
[2/25][801/79] Loss_D: 0.2411 Loss_G: 6.0013
[2/25][802/79] Loss_D: 0.1936 Loss_G: 5.2331
[2/25][803/79] Loss_D: 0.2676 Loss_G: 6.0872
[2/25][804/79] Loss_D: 0.2265 Loss_G: 5.0264
[2/25][805/79] Loss_D: 0.3541 Loss_G: 7.9256
[2/25][806/79] Loss_D: 0.4246 Loss_G: 3.9792
[2/25][807/79] Loss_D: 0.8997 Loss_G: 12.7910
[2/25][808/79] Loss_D: 1.5884 Loss_G: 8.1114
[2/25][809/79] Loss_D: 0.0551 Loss_G: 3.8815
[2/25][810/79] Loss_D: 0.9689 Loss_G: 10.7440
[2/25][811/79] Loss_D: 0.6678 Loss_G: 8.9908
[2/25][812/79] Loss_D: 0.0803 Loss_G: 5.8226
[2/25][813/79] Loss_D: 0.3944 Loss_G: 7.2173
[2/25][814/79] Loss_D: 0.2515 Loss_G: 5.4538
[2/25][815/79] Loss_D: 0.2715 Loss_G: 5.7375
[2/25][816/79] Loss_D: 0.2514 Loss_G: 5.2868
[2/25][817/79] Loss_D: 0.3033 Loss_G: 5.8403
[2/25][818/79] Loss_D: 0.2410 Loss_G: 4.4019
[2/25][819/79] Loss_D: 0.3012 Loss_G: 6.3560
[2/25][820/79] Loss_D: 0.2817 Loss_G: 4.0071
[2/25][8